In [1]:
#essential libraries

from google_drive import gdrive

from matplotlib.pyplot import show
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
import matplotlib
reload(matplotlib)
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy, copy
import random
from datetime import datetime, timedelta
from operator import itemgetter
from itertools import chain
import os
import gc
import linecache
import multiprocessing
from functools import partial
from itertools import repeat
from IPython.display import clear_output

import pprint
pp = pprint.PrettyPrinter()

import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef

from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy.cluster.hierarchy import fcluster
from scipy import spatial
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy import stats
from numpy.testing import assert_almost_equal

from scipy.spatial.distance import cosine as cosine_similarity

from divergence import gau_js as js_divergence
import building_tokenizer as toker
import brick_parser
reload(brick_parser)
from brick_parser import tagList, tagsetList, equipTagsetList, pointTagsetList, locationTagsetList,\
equalDict, pointTagList, equipTagList, locationTagList, equipPointDict
subTagListDict = dict([('point', pointTagList),
                          ('equip', equipTagList),
                          ('location', locationTagList)
                         ])
subTagsetListDict = dict([('point', pointTagsetList),
                          ('equip', equipTagsetList),
                          ('location', locationTagsetList)
                         ])

#from cmu_parser import cmu_building_parse

debugFlag = False

Visit here: https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.install&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=842684053512-91e0897dkl1iuiuunb0mkfkdqddi41fl.apps.googleusercontent.com&access_type=offline
4/gE6AcNyqyuF6gtPT4MRGFlMWOElsuzbKzw-gGEJOIuA
Authentication successful.
Init Graph
Load Brick.ttl
Load BrickFrame.ttl
Init Graph
Load Brick.ttl
Load BrickFrame.ttl


/home/jbkoh/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
workerNum = 7
coverageNum = 200
mpFlag = True

coverageNumDict = {'bonner':3213,
                   'ap_m':4380,
                   'ebu3b':4593
                  }

In [3]:
def save_fig(fig, name, dpi=400):
	pp = PdfPages(name)
	pp.savefig(fig, bbox_inches='tight', pad_inches=0, dpi=dpi)
	pp.close()

In [4]:
def pick_dict_with_best_n_from_dict(srcDict, n):
    idxCnt = 0
    sortedValueList = sorted(srcDict.values(), reverse=True)[0:n]
    chosenItemList = list()
    
    for score in sortedValueList:
        for key, val in srcDict.items():
            if idxCnt>=n:
                break
            if val==score:
                chosenItemList.append((key,val))
                idxCnt += 1
        if idxCnt>=n:
                break
    return OrderedDict(chosenItemList)

In [5]:
#from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o))#, file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

In [6]:
def get_equip_points(equip, g):
    pointList = list()
    hasPointRef = URIRef(uriPrefix+'hasPoint')
    if equip=='vav':
        equipRef = URIRef(uriPrefix+'VAV')
    elif equip=='ahu':
        equipRef = URIRef(uriPrefix+'AHU')
    elif equip=='vfd':
        equipRef = URIRef(uriPrefix+'VFD')
    elif equip=='chilled_water_pump':
        equipRef = URIRef(uriPrefix+'Chilled_Water_Pump')
    elif equip=='hot_water_pump':
        equipRef = URIRef(uriPrefix+'Hot_Water_Pump')
    elif equip=='crac':
        equipRef = URIRef(uriPrefix+'CRAC')
    elif equip=='exhaust_fan':
        equipRef = URIRef(uriPrefix+'AHU')
    elif equip=='return_fan':
        equipRef = URIRef(uriPrefix+'AHU')
    elif equip=='supply_fan':
        equipRef = URIRef(uriPrefix+'AHU')
    #elif equip=='return_fan':
    #    equipRef = URIRef(uriPrefix+'Return_Fan')
    #elif equip=='exhaust_fan':
    #    equipRef = URIRef(uriPrefix+'Exhaust_Fan')
    #elif equip=='supply_fan':
    #    equipRef = URIRef(uriPrefix+'Supply_Fan')
    else:
        print ("equip name %s is not in the schema" % equip)
        assert(False)
    for superClass in g.objects(equipRef,RDFS.subClassOf):
        #print triple
        #for tri in g.triples((triple, None, None)):
        if (superClass, OWL.onProperty, hasPointRef) in g:
            for point in g.objects(superClass, OWL.someValuesFrom):
                pointList.append(point.decode().split("#")[1].lower().split('_'))
    return pointList

def equip_ref_to_type(equipRef):
    if equipRef==None:
        return None
    equipRefList = equipRef.split('_')
    equipTypeList = list()
    for word in equipRefList:
        numList = re.findall('\d+', word)
        if len(numList)==0:
            equipTypeList.append(word)
    return str('_'.join(equipTypeList))

def equip_ref_to_ref(equipRef):
    if equipRef==None:
        return None
    equipRefList = equipRef.split('_')
    equipTypeList = list()
    for word in equipRefList:
        numList = re.findall('\d+', word)
        if len(numList)!=0:
            return word
    return '9999'

def get_alpha_word(word):
    alphaWordList = re.findall('[a-zA-Z]+', word)
    if len(alphaWordList)>0:
        alphaWord = alphaWordList[0]
        return alphaWord
    else:
        return ''

def get_num_word(word):
    wordList = re.findall('[b]?\d+[abc]?\_?\d*', word)
    if len(wordList)>0:
        foundWord = wordList[0]
        return foundWord
    else:
        return '9999'
    

In [7]:
class SentenceTagsetsCombination:
    totalWeightDict = None
    sentence = None
    
    def determine_word_belongs_to_sentence(self, word, tagList, scoreDictDict):
        #closestTagList = scoreDictDict[word].keys()[0].split()
        #return sum([1.0 if tag in tagList else 0.0 for tag in closestTagList]) / float(len(closestTagList)) >=0.2
        subScoreDict = pick_dict_with_best_n_from_dict(scoreDictDict[word], 3)
        totalScore = 0
        for tagset, score in subScoreDict.items():
            for tag in tagset.split():
                if tag in tagList:
                    totalScore += score / float(len(tagset.split()))
        #if word=='meter':
        #    print totalScore
        totalScore /= sum(subScoreDict.values())
        #return score>=0.2
        
        return totalScore>=0.2
    
    def __init__(self, sentence, subTagListDict, scoreDictDict, candSet=None):
        self.candSet = candSet
        self.scoreHistory = [0]
        self.sentence = sentence
        wordScoreDictDict = dict()
        for tagsetType in subTagListDict.keys()+['dummy']:
            wordScoreDictDict[tagsetType] = dict()
        
        for word in sentence:
            if len(word)<=1 or '' in scoreDictDict[word].keys():
                wordScoreDictDict['dummy'][word] = 1.0
                for tagsetType in subTagListDict.keys():
                    wordScoreDictDict[tagsetType][word] = 0.0
                continue
            
            wordOccurrenceDict = dict()
            for tagsetType, tagList in subTagListDict.items():
                wordOccurrenceDict[tagsetType] = 1.0 if self.determine_word_belongs_to_sentence(word, tagList, scoreDictDict) else 0.0
            totalOcc = sum(wordOccurrenceDict.values())
            
            #if word=='meter': ##########
            #    print wordOccurrenceDict
            
            if totalOcc==0:
                wordScoreDictDict['dummy'][word] = 1.0
                for tagsetType in wordOccurrenceDict.keys():
                    wordScoreDictDict[tagsetType][word] = 0.0
            else:
                wordScoreDictDict['dummy'][word]= 0.0
                for tagsetType, wordOccurrence in wordOccurrenceDict.items():
                    wordScoreDictDict[tagsetType][word] = wordOccurrence / totalOcc
        
        self.totalWeightDict = wordScoreDictDict
        
    def set_cand_set(self, candSet, score):
        selfTagsetTypes = set(self.totalWeightDict.keys())
        selfTagsetTypes.remove('dummy')
        
        assert(set(candSet.keys())==selfTagsetTypes)
        self.candSet = candSet
        self.scoreHistory.append(score)
        
    def set_score(self, score):
        self.scoreHistory.append(score)
    
    def get_score(self):
        return self.scoreHistory[-1]
    
    def get_cand_set(self):
        return self.candSet
        
    def set_sentence(self, sentence):
        self.sentence = sentence
        
    def get_sentence(self):
        return self.sentence
    
    def get_weight_dict(self, tagsetType):
        return self.totalWeightDict[tagsetType]
    
    def get_total_weight_dict(self):
        return self.totalWeightDict
    
    def set_total_weight_dict(self, newWeightDict):
        aWeightDict = self.totalWeightDict['dummy']
        assert(len(newWeightDict.keys())==len(self.totalWeightDict.keys()))
        for word in aWeightDict.keys():
            assert(np.allclose(1.0, sum([weightDict[word] for weightDict in newWeightDict.values()])))
        self.totalWeightDict = newWeightDict
        return self

In [8]:
class SubSentenceTagsetPair:
    localScoreDict = dict() # similar to scoreDictDict but only contains words existing in the sentence
    sentence = list()
    targetTagset = ''
    origSentence = list()
    predLable = ''
    score = 0 
    direction = ''
    scoreHistory = []
    
    def __init__(self, origSentence, localScoreDict, targetTagset, score=0):
        self.origSentence = origSentence
        self.sentence = origSentence
        self.localScoreDict = localScoreDict
        self.score = score
        self.scoreHistory = [score]
        self.targetTagset = targetTagset
    
    def append_score_history(self, score):
        self.scoreHistory.append(score)
    def get_score_history(self):
        return self.scoreHistory
    def set_score_dict(self, localScoreDict):
        self.localScoreDict = localScoreDict
    def set_direction(self, flag):
        if not (flag=='+' or flag=='-'):
            print 'wrong direction flag'
            assert(False)
        else:
            self.direction = flag
    def get_direction(self):
        return self.direction
    def get_score_dict(self):
        return self.localScoreDict
    def set_sentence(self, sentence):
        self.sentence = sentence
    def get_sentence(self):
        return self.sentence
    def set_score(self, score):
        self.score = score
    def get_score(self):
        return self.score
    def get_target_tagset(self):
        return self.targetTagset

In [9]:
buildingName = 'ap_m'
notUcsdBuildings = ['ghc']

tokenType = 'NoNumber'

if not buildingName in notUcsdBuildings:
    naeDict = dict()
    naeDict['bonner'] = ["607", "608", "609", "557", "610"]
    naeDict['ap_m'] = ['514', '513','604']
    naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
    naeDict['ebu3b'] = ["505", "506"]
    #naeDict['otterson'] = ["518", "517", "589", "590"]
    naeList = naeDict[buildingName]

    labeledFile = 'metadata/' + buildingName + '_sensor_types_location.csv'
    with open(labeledFile, 'rb') as fp:
        #truthDF = pd.read_excel(fp)
        truthDF = pd.DataFrame.from_csv(fp)
        #truthDF = truthDF.set_index(keys='Unique Identifier')

    wordFeatFile = 'data/wordfeat_'+buildingName+'.pkl'

    tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

    bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
    for val in Counter(unitMap.keys()).values():
        if val>1:
            "Unit map file ERROR"
            assert(False)
            
            
    trueDF = pd.DataFrame.from_csv('metadata/'+buildingName+'_sensor_types_location.csv')
    sensorDF, nameList, jcinameList, descList, unitList, bacnettypeList, wordList = \
    toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)

    origSensorDF = deepcopy(sensorDF)
    origNameList = deepcopy(nameList)
    origJcinameList = deepcopy(jcinameList)
    origDescList = deepcopy(descList)
    origUnitList = deepcopy(unitList)
    origBacnettypeList = deepcopy(bacnettypeList)
    origWordList = deepcopy(wordList)

    _, rawNameList, rawJcinameList, rawDescList, _, _, _ = toker.structure_metadata(buildingName=buildingName, tokenType='AlphaAndNum', \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)
    
    
else: 
    filename = 'metadata/'+buildingName+'_sensor_types_location.csv'
    #   filename = 'metadata/%s_sensor_types_location.csv'%buildingName
    df = pd.read_csv(filename)
    sentenceDict = dict()    
    for i,raw in enumerate(df['bas_raw'].tolist()):
        sentenceDict[i] = toker.tokenize(tokenType, raw)
    
    
## Common part
    
with open('data/'+buildingName+'_str_score_dict.pkl', 'rb') as fp:
    scoreDictDict = pickle.load(fp)
    ### If a word is exactly matched with another, fix it.
    for word, scoreDict in scoreDictDict.items():
        if word in scoreDict.keys():
            scoreDictDict[word] = {word:1}
    scoreDictDict['co'] = {'co2':1}
        
adder = lambda x,y:x+y
splitter = lambda x:x.split()
bacnetTypes = list(set(reduce(adder,map(splitter,origBacnettypeList),[])))
for bacnetType in bacnetTypes:
    scoreDictDict[bacnetType] = {bacnetType:1}
units = list(set(unitMap['word'].tolist()))
for unit in units:
    if type(unit)==str:
        scoreDictDict[unit] = {unit:1}

In [10]:
randNum = 100
randIdxList = random.sample(range(0,4000), randNum)
print randIdxList

[712, 3211, 1582, 145, 3780, 947, 724, 3360, 3728, 2115, 597, 2022, 823, 1273, 3182, 2254, 3150, 3652, 102, 21, 3756, 2480, 2795, 3294, 705, 2646, 2954, 2154, 2600, 2942, 440, 205, 1421, 204, 11, 861, 894, 2234, 2615, 3563, 3033, 2511, 3943, 3719, 2311, 3196, 448, 880, 2527, 2196, 579, 2365, 3902, 2775, 3881, 1878, 3880, 1688, 3074, 1910, 3651, 1574, 2689, 379, 1253, 1474, 3268, 2268, 3462, 2018, 1821, 1767, 1860, 1381, 2722, 3821, 1415, 1692, 1028, 1223, 2356, 1383, 1359, 2984, 1966, 450, 273, 1358, 3092, 3210, 3863, 31, 718, 3664, 2996, 771, 1754, 1350, 3435, 2394]


In [11]:
#print randIdxList
randIdxList = sorted([224, 3226, 2175, 3474, 602, 146, 1813, 640, 1055, 3778, 2276, 1019, 390, 2208, 3301, 1090, 2089, 10, 1143, 2113, 1927, 3325, 2533, 1098, 1777, 3352, 710, 1765, 3228, 59, 2867, 2273, 3551, 269, 1119, 2373, 3647, 330, 2983, 1994, 2593, 2784, 3578, 786, 3574, 1038, 2826, 613, 3786, 2862, 2435, 2847, 2693, 2442, 3554, 1750, 163, 412, 3746, 3619, 3114, 3565, 1970, 636, 1338, 1226, 2100, 566, 2553, 303, 1857, 2960, 652, 448, 3333, 600, 47, 1762, 633, 263, 2564, 2069, 525, 396, 749, 3177, 3587, 2719, 2967, 3336, 2706, 2515, 1823, 2242, 1824, 866, 82, 2078, 1799, 2604])
print randIdxList

[10, 47, 59, 82, 146, 163, 224, 263, 269, 303, 330, 390, 396, 412, 448, 525, 566, 600, 602, 613, 633, 636, 640, 652, 710, 749, 786, 866, 1019, 1038, 1055, 1090, 1098, 1119, 1143, 1226, 1338, 1750, 1762, 1765, 1777, 1799, 1813, 1823, 1824, 1857, 1927, 1970, 1994, 2069, 2078, 2089, 2100, 2113, 2175, 2208, 2242, 2273, 2276, 2373, 2435, 2442, 2515, 2533, 2553, 2564, 2593, 2604, 2693, 2706, 2719, 2784, 2826, 2847, 2862, 2867, 2960, 2967, 2983, 3114, 3177, 3226, 3228, 3301, 3325, 3333, 3336, 3352, 3474, 3551, 3554, 3565, 3574, 3578, 3587, 3619, 3647, 3746, 3778, 3786]


In [12]:
len(sensorDF)

4380

In [13]:
randSrcidList = list()
for randIdx in randIdxList:
    if randIdx <len(sensorDF):
        srcid = sensorDF.iloc[randIdx].name
    randSrcidList.append(srcid)
print randSrcidList

['514_0_3006443', '514_0_3006661', '514_0_3006711', '514_0_3006822', '514_0_3007073', '514_0_3007151', '514_0_3007388', '514_0_3007528', '514_0_3007553', '514_0_3011251', '514_1_3006410', '514_1_3006647', '514_1_3006658', '514_1_3006699', '514_1_3006791', '514_1_3007025', '514_1_3007140', '514_1_3007267', '514_1_3007269', '514_1_3007304', '514_1_3007372', '514_1_3007377', '514_1_3007393', '514_1_3007438', '514_1_3011258', '514_2_3010337', '514_3_3007491', '514_4_3007474', '513_0_3005109', '513_0_3005200', '513_0_3005272', '513_0_3005401', '513_0_3005420', '513_0_3005492', '513_0_3005621', '513_0_3006007', '513_0_3006493', '513_1_3004779', '513_1_3004816', '513_1_3004820', '513_1_3004848', '513_1_3004909', '513_1_3004940', '513_1_3004964', '513_1_3004968', '513_1_3005054', '513_1_3005234', '513_1_3005346', '513_1_3005524', '513_1_3005727', '513_1_3005750', '513_1_3005775', '513_1_3005811', '513_1_3005841', '513_1_3006004', '513_1_3006091', '513_1_3006180', '513_1_3006256', '513_1_300626

In [14]:
idxList = range(0,len(trueDF))
listIndexFunc = lambda l, i: [l[ii] for ii in i]
sensorDF = origSensorDF.iloc[idxList]
nameList = listIndexFunc(origNameList, idxList)
jcinameList = listIndexFunc(origJcinameList, idxList)
descList = listIndexFunc(origDescList, idxList)
unitList = listIndexFunc(origUnitList, idxList)
bacnettypeList = listIndexFunc(origBacnettypeList, idxList)


In [15]:
def calc_max_score_dict(scoreDictDict):
    maxScoreDict = dict()
    for word, scoreDict in scoreDictDict.items():
        maxScoreDict[word] = max(scoreDict.values())
    return maxScoreDict


In [16]:
def manual_def(word, trueTagset):
    scoreDict = scoreDictDict[word]
    scoreDictDict[word] = dict((tagset,1) if turTagset==tagset else (tagset,0) for tagset in scoreDict.keys())

In [17]:
### Initialization sentence separation
#### We need equip sentence
### We need type sentence, equip sentence, location sentence

# Init no meaning sentence
sentenceDict = list()
adder = lambda a,b:a+b
splitter = lambda s:s.split()
sentenceDict = dict([(i,reduce(adder, map(splitter, dataList)))\
                for i,dataList in enumerate(zip(nameList, jcinameList, descList, unitList, bacnettypeList))])
                #for dataList in zip(nameList, jcinameList, unitList, bacnettypeList)]
                #for dataList in zip(nameList, jcinameList, unitList)]
                #for dataList in zip(nameList, jcinameList)]
rawSentenceDict = dict([(i,reduce(adder, map(splitter, dataList)))\
                for i,dataList in enumerate(zip(rawNameList, rawJcinameList, rawDescList, unitList, bacnettypeList))])
                #for dataList in zip(rawNameList, rawJcinameList, unitList, bacnettypeList)]
                #for dataList in zip(rawNameList, rawJcinameList, unitList)]
                #for dataList in zip(rawNameList, rawJcinameList)]

# Make type, equip, location sentences
equipSentenceList = list()
pointSentenceList = list()
locationSentenceList = list()

for i, sentence in sentenceDict.items():
    equipSentence = list()
    pointSentence = list()
    locationSentence = list()
    for word in sentence:
        closestTagList = scoreDictDict[word].keys()[0].split()
        if len(closestTagList)==0:
            continue
        if sum([1 if tag in equipTagList else 0 for tag in closestTagList]) / float(len(closestTagList)) >=0.3:
            equipSentence.append(word)
        if sum([1 if tag in pointTagList else 0 for tag in closestTagList]) / float(len(closestTagList)) >=0.3:
            pointSentence.append(word)
        if sum([1 if tag in locationTagList else 0 for tag in closestTagList]) / float(len(closestTagList)) >=0.3:
            locationSentence.append(word)
    equipSentenceList.append(equipSentence)
    pointSentenceList.append(pointSentence)
    locationSentenceList.append(locationSentence)


origSentenceDict = deepcopy(sentenceDict)

In [18]:
idxList = range(0,coverageNum)
sentenceDict = dict((idx,sentence) for idx,sentence in origSentenceDict.items() if idx in idxList)
sentenceDict = dict((idx, origSentenceDict[idx]) for idx in idxList)

In [19]:
### Initial Human Input

maxScoreDict = calc_max_score_dict(scoreDictDict)

adder = lambda x,y:x+y
wordList = list(set(reduce(adder,origSentenceDict.values(), [])))
i = 0
for word in wordList:
    if sum([1 if word in sentence else 0 for sentence in origSentenceDict.values()])>len(origSentenceDict)*0.2 and len(word)>1:
        if len(scoreDictDict[word].values())>1:
            print word
            i += 1
print '=========='
print 'total default asking: ', i

trunk
rm
ap
stpt
nae
total default asking:  5


In [20]:
if buildingName=='bonner':
    scoreDictDict['th'] = {'':1}
    scoreDictDict['flr'] = {'':1}
    scoreDictDict['vma'] = {'':1}
    scoreDictDict['rm'] = {'room':1}
    scoreDictDict['fc'] = {'':1}
    scoreDictDict['bonner'] = {'':1}
    scoreDictDict['sav'] = {'vav':1}
    scoreDictDict['ah'] = {'ahu':1}
    scoreDictDict['nae'] = {'':1}
    scoreDictDict['nd'] = {'':1}
elif buildingName=='ebu3b':
    scoreDictDict['ebu'] = {'':1}
    scoreDictDict['vma'] = {'':1}
    scoreDictDict['nae'] = {'':1}
    scoreDictDict['rm'] = {'room':1}
elif buildingName=='ap_m':
    scoreDictDict['ap'] = {'':1}
    scoreDictDict['trunk'] = {'':1}
    scoreDictDict['rm'] = {'room':1}
    scoreDictDict['stpt'] = {'setpoint':1}
    scoreDictDict['nae'] = {'':1}


#scoreDictDict['actual'] = {'effective':1}
#scoreDictDict['percent'] = {'':1}

maxScoreDict = calc_max_score_dict(scoreDictDict)

origScoreDictDict = deepcopy(scoreDictDict)

In [21]:
def calc_tfidf_sub(cntDict, idfDict):
    tfidfList = list()
    maxCnt = max(cntDict.values())
    for tag, idf in idfDict.items():
        if not tag in cntDict.keys():
            tfidf = 0
        else:
            tfidf = (0.5 + 0.5 * cntDict[tag] / maxCnt)* idf
        tfidfList.append(tfidf)
    return np.asarray(tfidfList)

def sentence_tagset_score_segmented_tfidf(sentence, targetTagset):
    targetTagset = targetTagset.split('_')
    compDataLen = 5
    sentenceCntDict = defaultdict(float)
    
    for word in sentence:
        totalWordScore = 1.0/len(sentence)
        scoreDict = dict(scoreDictDict[word].items()[0:compDataLen])
        localScoreSum = float(sum(scoreDict.values())) #* len(sentence)
        for candTagset, score in scoreDict.items():
            candTagList = candTagset.split()
            candTagScore = score / localScoreSum / len(candTagList)
            for candTag in candTagList:
                sentenceCntDict[candTag] += candTagScore
                
    tagsetCntDict = defaultdict(float)
    #avgCnt = np.mean(sentenceCntDict.values())
    for tag in targetTagset:
        tagsetCntDict[tag] += 1.0/len(targetTagset)
        
    entireCntDict = OrderedDict(sentenceCntDict)
    for tag in targetTagset:
        if tag not in sentenceCntDict.keys():
            entireCntDict[tag] = tagsetCntDict[tag]
    
    entireCntDict = OrderedDict(entireCntDict)
    totalCnt = sum(entireCntDict.values())
    idfValues = [np.log(totalCnt/cnt) for cnt in entireCntDict.values()]
    idfDict = OrderedDict()
    for tag, idf in zip(entireCntDict.keys(), idfValues):
        idfDict[tag]  = idf
    
    sentenceTfidf = calc_tfidf_sub(sentenceCntDict, idfDict)
    tagsetTfidf = calc_tfidf_sub(tagsetCntDict, idfDict)
    return 1 - cosine_similarity(sentenceTfidf, tagsetTfidf)

#sentence_tagset_score = sentence_tagset_score_segmented_tfidf

In [22]:
# Maybe need log-scale compensation of redundant tags or words

needDataLen =12

def sentence_tagset_score_word_and_sentence_together(sentence, targetTagset, scoreDictDict=scoreDictDict):
    origSentence = sentence
    if type(targetTagset)==str or type(targetTagset)==unicode:
        targetTagset = targetTagset.split('_')
    
    totalScore = 0 
    notUsedWordCntDict = dict()
    for word in sentence:
        notUsedWordCntDict[word] = 0
        
    tagsetCntDict = dict()
    for tag in targetTagset:
        tagsetCntDict[tag] = 0
    
    for word in sentence:
        scoreDict = dict(scoreDictDict[word].items()[0:needDataLen])
        localScoreSum = float(sum(scoreDict.values()))
        for candTagset, score in scoreDict.items():
            candTags = candTagset.split()
            for candTag in candTags:
                candTagScore = score / localScoreSum / len(candTags)
                if candTag in targetTagset:
                    tagsetCntDict[candTag] += candTagScore
                else:
                    notUsedWordCntDict[word] += candTagScore
    tagsetScoreDict = dict()
    for word, cnt in tagsetCntDict.items():
        tagsetScoreDict[word] = np.log(1+cnt)
    return (len(sentence)-sum(notUsedWordCntDict.values()))/len(sentence) * sum(tagsetScoreDict.values()) / len(targetTagset)
    
sentence_tagset_score = sentence_tagset_score_word_and_sentence_together

In [23]:
def sentence_tagset_score_with_weight_inside(sentence, targetTagset, weightDict, scoreDictDict=scoreDictDict, \
                                             maxScoreDict=maxScoreDict, totalScore=0, sentenceLen=None):
    if type(targetTagset)==str or type(targetTagset)==unicode:
        targetTagset = targetTagset.split('_')
    
    #TODO: Choose between those two.
    if sentenceLen==None:
        sentenceLen = len(sentence)
    
    totalScoreForTagset = 0
    for word in sentence:
        totalScoreForTagset += weightDict[word]
    
    try:
        assert_almost_equal(totalScoreForTagset,0)
        return 0, 0, 0    
    except:
        pass
        
    
    for word in deepcopy(sentence):
        if len(word)<=1:
            sentence.remove(word)
    
    notUsedWordCntDict = dict()
    for word in sentence:
        notUsedWordCntDict[word] = 0
        
    tagsetCntDict = dict()
    for tag in targetTagset:
        tagsetCntDict[tag] = 0
        
    sysWordUsageDict = defaultdict(float)
    
    for word in sentence:
        wordScore = weightDict[word] * max(scoreDictDict[word].values()) / maxScoreDict[word]
        
        if wordScore==0:
            continue
            
        scoreDict = dict(scoreDictDict[word].items()[0:needDataLen])
        localScoreSum = float(sum(scoreDict.values()))
        for candTagset, score in scoreDict.items():
            candTags = candTagset.split()
            for candTag in candTags:
                candTagScore = score / localScoreSum / len(candTags) * wordScore
                if candTag in targetTagset:
                    tagsetCntDict[candTag] += candTagScore
                else:
                    notUsedWordCntDict[word] += candTagScore
    tagsetScoreDict = dict()
    for tag, cnt in tagsetCntDict.items():
        tagsetScoreDict[tag] = np.log(1+cnt)
    #for tag, cnt in tagsetCntDict.items():
    #    tagsetScoreDict[tag] = cnt / (totalScoreForTagset/float(len(targetTagset)))
    
    
    notUsedWordScoreDict = dict()
    for word, cnt in notUsedWordCntDict.items():
        notUsedWordScoreDict[word] = np.log(1+cnt)
    #print "notusedscore: ", notUsedWordScoreDict['chwp']
        
    
    
    sentenceUsage = sum(tagsetCntDict.values()) / float(sentenceLen)
    tagsetCoverage = sum(tagsetScoreDict.values()) / (len(targetTagset)*np.log(1+totalScoreForTagset/float(len(targetTagset))))
    #tagsetCoverage = sum(tagsetCntDict.values()) / totalScoreForTagset
    candScore = sentenceUsage  * tagsetCoverage
    
    #print "---", targetTagset
    #print "totalScoreForTagset", totalScoreForTagset
    #print "len(targetTagset)", len(targetTagset)
    #print "---"
    
    
    #print sentenceUsage
    #print tagsetCoverage
    #pp.pprint(tagsetScoreDict)
    
    
    return candScore, sentenceUsage, tagsetCoverage#, notUsedWordCntDict
    
    
def sentence_tagset_score_with_weight(sentence, targetTagset, weightDict, scoreDictDict=scoreDictDict, maxScoreDict=maxScoreDict, totalScore=0):
    candScore, _, _ = sentence_tagset_score_with_weight_inside(sentence, targetTagset, weightDict, scoreDictDict=scoreDictDict, maxScoreDict=maxScoreDict, totalScore=0)
    return candScore

In [24]:
def calc_score_altogether_in_sentence(comb, candDict, scoreDictDict):
    sentence = comb.get_sentence()
    #TODO: Check which is good between below two lines
    #sentenceTotalScore = len(sentence) - sum(comb.get_weight_dict('dummy').values())
    sentenceTotalScore = len(sentence)
    sentenceUsageSum = 0
    tagsetCoverageList = list()
    weightSumList = list()
    candScoreSum = 0
    #TODO: Choose to include this or not
    #sentenceLen = len(sentence) - sum(comb.get_weight_dict('dummy').values())
    sentenceLen = None
    for tagsetType, cand in candDict.items():
        # TODO: Return to the original
        candScore, sentenceUsage, tagsetCoverage = sentence_tagset_score_with_weight_inside(sentence, cand, \
                                        comb.get_weight_dict(tagsetType), scoreDictDict, sentenceLen=sentenceLen)
        sentenceUsageSum += sentenceUsage
        tagsetCoverageList.append(tagsetCoverage)
        weightSumList.append(sum(comb.get_weight_dict(tagsetType).values())) 
        candScoreSum += candScore
        #print cand, candScore, sentenceUsage, tagsetCoverage
    #sentenceUsage = sentenceUsageSum / sentenceTotalScore
    #totalTagsetCoverage = sum([cov*weight for cov, weight in zip(tagsetCoverageList, weightSumList)])/float(sum(weightSumList))
    return candScoreSum

In [25]:
extractSubDict = lambda origDict, tagsetTypeList: \
            dict((tagsetType, subTagList) for tagsetType, subTagList in origDict.items() if tagsetType in tagsetTypeList)
subTagListDictList = [extractSubDict(subTagListDict, ['point']),\
                      extractSubDict(subTagListDict, ['point', 'equip']),\
                      extractSubDict(subTagListDict, ['point', 'equip', 'location']),\
                      extractSubDict(subTagListDict, ['point', 'location'])
                      ]
                      
scoreDictDict = dict()



for idx, sentence in sentenceDict.items():    
    for word in sentence:
        if word not in scoreDictDict.keys():
            scoreDictDict[word] = origScoreDictDict[word]

In [26]:
def init_comb((sentence, subTagListDictList, scoreDictDict, reduceCandNum)):
    combList = list()
    for tagListDict in subTagListDictList:
        tagTypeList = tagListDict.keys()
        comb = SentenceTagsetsCombination(sentence, tagListDict, scoreDictDict)
        sentenceTotalScore = len(comb.get_sentence()) - sum(comb.get_weight_dict('dummy').values())
        weightDictDict = comb.get_total_weight_dict()
        candDictList = list()
        multipleTagsetScoreDict = defaultdict(dict)
        for tagsetType, weightDict in weightDictDict.items():
            if tagsetType=='dummy':
                continue
            tagsetList = subTagsetListDict[tagsetType]
            for tagset in tagsetList:
                #multipleTagsetScoreDict[tagsetType][tagset] = sentence_tagset_score_with_weight(sentence, \
#                                                                                                tagset, weightDict)
                multipleTagsetScoreDict[tagsetType][tagset] = calc_score_altogether_in_sentence(comb, {tagsetType:tagset}, \
                                                                                                scoreDictDict)
        multipleTagsetScoreDict = dict(multipleTagsetScoreDict)
        
        pointItemList = list()
        equipPointItemList = list()
        
        pointCandNum = 5 - reduceCandNum
        if pointCandNum<1:
            pointCandNum = 1
        
        pointEquipCandNum = 5 - reduceCandNum
        if pointEquipCandNum<2:
            pointEquipCandNum = 2
        locationCandNum = 3 - reduceCandNum
        if locationCandNum <2:
            locationcandNum = 2
        
        chosenPointTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['point'], pointCandNum)
        #chosenItemList 
        pointItemList += [({'point':pointTagset}, score) for pointTagset, score in chosenPointTagsetDict.items()]
        
        equipPointItemList = list()
        
        if 'equip' in tagTypeList:
            chosenEquipTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['equip'], pointEquipCandNum)
            for equipTagset, equipScore in chosenEquipTagsetDict.items():
                for pointTagset, pointScore in chosenPointTagsetDict.items():
                    if pointTagset in equipPointDict[equipTagset]:
                        candDict = {'point': pointTagset, 'equip':equipTagset}
                        equipPointItemList.append((candDict, calc_score_altogether_in_sentence(comb, candDict, scoreDictDict)))
        '''
        if 'equip' in tagTypeList:
            chosenEquipTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['equip'], 4 - reduceCandNum)
            for equipTagset, equipScore in chosenEquipTagsetDict.items():
                for pointTagset, pointScore in chosenPointTagsetDict.items():
                    candDict = {'point': pointTagset, 'equip':equipTagset}
                    equipPointItemList.append((candDict, calc_score_altogether_in_sentence(comb, candDict, scoreDictDict)))
        '''
        
        if 'location' in tagTypeList:
            chosenLocationTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['location'], locationCandNum)
            locationEquipPointItemList = list()
            locationPointItemList = list()
            for locationTagset, locationScore in chosenLocationTagsetDict.items():
                if 'equip' in tagTypeList:
                    for item in equipPointItemList:
                        item[0]['location'] = locationTagset
                        locationEquipPointItemList.append((deepcopy(item[0]), item[1]+locationScore))
                else:
                    for item in pointItemList:
                        item[0]['location'] = locationTagset
                        locationPointItemList.append((deepcopy(item[0]), item[1]+locationScore))
                    
                    
        if 'equip' in tagTypeList:
            if 'location' in tagTypeList:
                chosenItemList = locationEquipPointItemList
            else:
                chosenItemList = equipPointItemList
        elif 'location' in tagTypeList:
            chosenItemList = locationPointItemList
        else:
            chosenItemList = pointItemList
        for item in chosenItemList:
            tempComb = deepcopy(comb)
            tempComb.set_cand_set(item[0], item[1])
            combList.append(tempComb)
    return combList

In [27]:
beginTime =  datetime.now()
if mpFlag:
    pool = multiprocessing.Pool(workerNum)
    combListList = pool.map(init_comb, zip(sentenceDict.values(), repeat(subTagListDictList), repeat(scoreDictDict), repeat(0)))
    pool.terminate()
    pool.close()
    pool.join()
else:
    #combListList = map(init_comb, zip(sentenceList, repeat(subTagListDictList), repeat(scoreDictDict), repeat(0)))
    combListList = list()
    for i, sentence in enumerate(sentenceDict.values()):
        combListList.append(init_comb((sentence, subTagListDictList, scoreDictDict, 0)))
    pass
combListDict = dict((idx,combList) for idx, combList in zip(idxList, combListList))
endTime = datetime.now()

origCombListDict = deepcopy(combListDict)

print (endTime-beginTime).total_seconds()/60, 'minutes'

0.717161266667 minutes


In [28]:
def update_comb_cand_dict_score(combListDict, scoreDictDict):
    for idx, combList in combListDict.items():
        for comb in combList:
            score = calc_score_altogether_in_sentence(comb, comb.get_cand_set(), scoreDictDict)
            comb.set_score(score)
    return combListDict

In [29]:
def calc_building_score(combListDict):
    buildingScore = 0
    scoreGetter = lambda x: x.get_score()
    for idx, combList in combListDict.items():
        scoreList = map(scoreGetter,combList)
        scoreSum = sum(scoreList)
        buildingScore += sum([score*score/scoreSum for score in scoreList])
        #buildingScore += sum([score*score/scoreSum/scoreSum for score in scoreList])
        #buildingScore += stats.entropy(scoreList)
        
    return buildingScore

buildingScoreList = [calc_building_score(origCombListDict)]
print buildingScoreList

[47.109588090145898]


In [30]:
def eval_word_meaning2((combListDict, word, scoreDictDict)):
    scoreDict = scoreDictDict[word]
    if len(scoreDict)<=1:
        return None
    baseCombListDict = dict()
    for i, combList in combListDict.items():
        if word in combList[0].get_sentence():
            #baseCombListDict[i] = deepcopy(combList)
            baseCombListDict[i] = combList
            #TODO: Rollback if not works
    if len(baseCombListDict)<1:
        return None
    baseBuildingScore = calc_building_score(baseCombListDict)
    
    removeCandList = scoreDict.keys()
    #updatedCombListDictList = list()
    updatedScoreList = list()
    for removeCand in removeCandList:
        updatedScoreDictDict = deepcopy(scoreDictDict)
        del updatedScoreDictDict[word][removeCand]
        
        #combListDict = deepcopy(baseCombListDict)
        
        #updatedCombListDictList.append(update_comb_cand_dict_score(combListDict, scoreDictDict=updatedScoreDictDict))
        #updatedCombListDict = update_comb_cand_dict_score(combListDict, scoreDictDict=updatedScoreDictDict)
        updatedCombListDict = update_comb_cand_dict_score(baseCombListDict, scoreDictDict=updatedScoreDictDict)
        updatedScoreList.append(calc_building_score(updatedCombListDict))
        combListDict = None
        updatedCombListDict = None
    
    #for updatedCombListDict in updatedCombListDictList:
    #    updatedScoreList.append(calc_building_score(updatedCombListDict))
    
    baseCombListDict = None
    combListDict = None
    updatedScoreDictDict = None
    if max(updatedScoreList) > baseBuildingScore:
        selectedScore = max([score for score in updatedScoreList if [score - baseBuildingScore]>0])
        removeTagset = removeCandList[updatedScoreList.index(selectedScore)]
        return (word, removeTagset)
    else:
        return None
    
wordUpdateIterationNum = 0

In [31]:
def try_add_weight(word, targetTagsetType, totalWeightDict, delta=0.1):
    tagsetTypeTotalNum = len(totalWeightDict)
    totalWeightDict[targetTagsetType][word] += delta
    currWeight = totalWeightDict[targetTagsetType][word]
    if currWeight>1:
        delta -= currWeight - 1
        totalWeightDict[targetTagsetType][word] = 1
    returnWeight = 0
    for tagsetType in totalWeightDict.keys():
        if tagsetType==targetTagsetType:
            continue
        totalWeightDict[tagsetType][word] -= delta / float((tagsetTypeTotalNum-1))
        if totalWeightDict[tagsetType][word]<0:
            returnWeight += (0-totalWeightDict[tagsetType][word])
            totalWeightDict[tagsetType][word] = 0
    totalWeightDict[targetTagsetType][word] -= returnWeight
    return totalWeightDict

In [32]:
def eval_weight_per_comb(comb, scoreDictDict):
    delta = 0.1
    candSet = comb.get_cand_set()
    origTotalWeightDict = comb.get_total_weight_dict()
    words = origTotalWeightDict.values()[0].keys()
    foundTotalWeightDict = defaultdict(dict)
    tagsetTypeTotalNum = len(origTotalWeightDict)
    for word in words:
        totalWeightDictCandDict = dict()
        scoreDict = dict()
        for targetTagsetType, weightDict in origTotalWeightDict.items():
            totalWeightDict = try_add_weight(word, targetTagsetType, deepcopy(origTotalWeightDict), delta=delta)
            score = calc_score_altogether_in_sentence(comb.set_total_weight_dict(totalWeightDict), candSet, scoreDictDict)
            scoreDict[targetTagsetType] = score
            totalWeightDictCandDict[targetTagsetType] = totalWeightDict
        
        maxScore = max(scoreDict.values())
        maxIdx = scoreDict.values().index(maxScore)
        maxTagsetType = scoreDict.keys()[maxIdx]
        
        chosenTotalWeightDict = totalWeightDictCandDict[maxTagsetType]
        for tagsetType in totalWeightDict.keys():
            foundTotalWeightDict[tagsetType][word] = chosenTotalWeightDict[tagsetType][word]
    comb.set_total_weight_dict(foundTotalWeightDict)
    comb.set_score(calc_score_altogether_in_sentence(comb, candSet, scoreDictDict))
    return comb

def eval_weight((combListDict, scoreDictDict)):
    newCombListDict = defaultdict(list)
    for idx, combList in combListDict.items():
        for comb in combList:
            newCombListDict[idx].append(eval_weight_per_comb(comb, scoreDictDict))
    return dict(newCombListDict)

In [33]:
def remove_weak_candidates(combListDict):
    scoreGetter = lambda x:x.get_score()
    for idx, combList in combListDict.items():
        if len(combList)<4:
            continue
        scoreList = map(scoreGetter, combList)
        minVal = min(scoreList)
        minIdx = scoreList.index(minVal)
        del combList[minIdx]
        combListDict[idx] = combList
    return combListDict

In [34]:
#### Asking words' meaning to experts
def change_word_meaning(word, meaning, scoreDictDict=scoreDictDict, maxScoreDict=maxScoreDict):
    scoreDictDict[word] = {meaning: maxScoreDict[word]}
    return scoreDictDict

def pick_weak_word_in_cluster(idxList, combListDict, scoreDictDict):
    adder = lambda x,y:x+y
    wordOccurrenceDict = dict()
    wordUsageDict = dict()
    occIdxDict = dict()
    cnt = 0
    wordEntropyDict = dict()
    candSetGetter = lambda x:x.get_cand_set()
    scoreGetter = lambda x:x.get_score()
    for word, scoreDict in scoreDictDict.items():
        wordEntropyDict[word] = stats.entropy(scoreDict.values())
        cnt += 1
        if cnt%100==0:
            print cnt
        occIndices = [idx for idx, sentence in zip(idxList, [sentenceDict[i] for i in idxList]) if word in sentence]
        occIdxDict[word] = occIndices
        wordOccurrence = len(occIndices)
        wordOccurrenceDict[word] = wordOccurrence
        if wordOccurrence==0 or len(scoreDict)==1:
            continue
        usageScore = 0
        for idx in occIndices:
            combList = combListDict[idx]
            #candSetList = map(candSetGetter, combList)
            totalScore = sum(map(scoreGetter, combList))
            subScore = 0
            wordCandScoreSum = sum(scoreDict.values())
            for wordTagsetCand, tagsetScore in scoreDict.items():
                wordTagsetCand = wordTagsetCand.split()
                for tagCand in wordTagsetCand:
                    for comb in combList:
                        for cand in comb.get_cand_set().values():
                            cand = cand.split('_')
                            if tagCand in cand:
                                subScore += tagsetScore/wordCandScoreSum/len(wordTagsetCand) * comb.get_score() / totalScore / len(cand)
            usageScore += subScore
        wordUsageDict[word] = usageScore

    percentUsageDict = dict([(word, wordUsageDict[word]/wordOccurrenceDict[word]) for word in wordUsageDict.keys()])

    sortedUsage = sorted(percentUsageDict.values())
    '''print "========"
    for word, usage in sorted(percentUsageDict.items(), key=itemgetter(1)):
        if wordOccurrenceDict[word]>2 and len(word)>1:
            print word, usage, wordOccurrenceDict[word], wordEntropyDict[word]
    print "========"
    '''
    return percentUsageDict

#percentUsageDict = pick_weak_word_in_cluster(range(0,coverageNum), combListDict, scoreDictDict)


def ask_experts(bestCandScoreDict, combListDict, scoreDictDict=scoreDictDict, maxScoreDict=maxScoreDict):
    answerCnt = 0
    
    percentUsageDict = pick_weak_word_in_cluster(range(0,coverageNum), combListDict, scoreDictDict)
    
    pointIdxDict = defaultdict(list)
    for i, (bestComb, bestCand, bestScore) in bestCandScoreDict.items():
        pointIdxDict[bestCand['point']].append(i)
    avgScorePerPointDict = dict()
    for key, idxList in pointIdxDict.items():
        avgScorePerPointDict[key] = np.mean([cand[2] for cand in [bestCandScoreDict[i] for i in idxList]])
    for key, mean in sorted(avgScorePerPointDict.items(), key=itemgetter(1)):
        #print key, mean, len(pointIdxDict[key])
        pass

    lenGetter = lambda x: len(x[1])
    totalScoreGetter = lambda (key,avg):avg*len(pointIdxDict[key])

    weakestPointList = list()
    #for key, idxList in sorted(pointIdxDict.items(), key=lenGetter, reverse=True):
    #    print key, len(pointIdxDict[key])
    for key, avg in sorted(avgScorePerPointDict.items(), key=totalScoreGetter):
        num = len(pointIdxDict[key])
        if num>3:
            weakestPointList.append(key)
            
    #print "=================score baesd asking"
    #for pointTagset in weakestPointList[0:15]:
    #    print pick_weak_word_in_cluster(pointIdxDict[pointTagset], candListDict, scoreDictDict)
    #print "\n"
        
        
    commonWordListDict = defaultdict(set)
    adder = lambda x,y:x+y
    for pointTagset, idxList in pointIdxDict.items():        
        subSentenceList = [sentenceDict[i] for i in idxList]
        allWordList = list(set(reduce(adder, subSentenceList,[])))
        for word in allWordList:
            cnt = 0
            for sentence in subSentenceList:
                if word in sentence:
                    cnt += 1
                if cnt/float(len(idxList)) > 0.25:
                    commonWordListDict[pointTagset].add(word)
    
    avgScoreGetter = lambda item: avgScorePerPointDict[item[0]]
    commonWordListDict = OrderedDict(sorted(commonWordListDict.items(), key=avgScoreGetter))
    
    shownWordList = list()
    anchorListDict = defaultdict(list)
    for pointTagset, commonWordList in commonWordListDict.items():
        if pointIdxDict[pointTagset]<5:
            continue
        for word in commonWordList:
            if word not in shownWordList:
                if word in percentUsageDict.keys() and len(word)>1:
                    print word, percentUsageDict[word]
                    shownWordList.append(word)
                    anchorListDict[pointTagset].append((word, percentUsageDict[word]))
    
    itemNumGetter = lambda x:len(pointIdxDict[x[0]])

    for pointTagset, anchorList in sorted(anchorListDict.items(), key=itemNumGetter, reverse=True):
        print pointTagset
        print '\t', sorted(anchorList, key=itemgetter(1))
        
    resp = ''
    while resp!='q':
        resp = raw_input('''
if you want to fix words, put a word to fix.
stop asking for now and init again?\t q
stop asking for now but no init?\t qq
stop asking forever and no init?\t qqq
                            ''')
        if resp=='q':
            pass
        elif resp=='qq':
            return True, False, answerCnt
        elif resp=='qqq':
            return False, False, answerCnt
        else:
            word = resp
            if word not in scoreDictDict.keys():
                print "What was the word again?"
                continue
            meaning = raw_input("meaning?\n")
            change_word_meaning(word, meaning, scoreDictDict)
            answerCnt += 1
    return True, True, answerCnt

In [35]:
def calc_best_cand_dict(combListDict):
    bestCandScoreDict = dict()
    scoreGetter = lambda x:x.get_score()
    for i, combList in combListDict.items():
        scoreList = map(scoreGetter, combList)
        maxVal = max(scoreList)
        maxIdx = scoreList.index(maxVal)
        bestComb = combList[maxIdx]
        bestCandScoreDict[i] = (bestComb, bestComb.get_cand_set(), maxVal)
    return bestCandScoreDict

In [39]:
def check_correct(predLabel, currLabel):
    currLabel = currLabel.replace(' ', '_')
    possibleLabelList = [currLabel]
    if 'supply' in currLabel:
        possibleLabelList.append(currLabel.replace('supply', 'discharge'))
    if 'discharge' in currLabel:
        possibleLabelList.append(currLabel.replace('discharge', 'supply'))
    if currLabel in equalDict.keys():
        possibleLabelList.append(equalDict[currLabel])
    if predLabel in possibleLabelList:
        return True
    else:
        return False

In [40]:
def str_conv(d):
    if type(d)==str:
        return d
    else:
        if np.isnan(d):
            return ''
        else:
            return str(d)
        
def eval_result(bestCandScoreDict, randIdxList, trueDF):
    cnt = 0
    totalCntDict = defaultdict(int)
    correctCntDict = defaultdict(int)
    incorrectCntDict = defaultdict(int)
    unknownDict = defaultdict(int)
    randomSelectedDict = dict()
    for i, (comb, candSet, candSetScore) in bestCandScoreDict.items():
        if i in randIdxList:
            cnt += 1
            srcid = sensorDF.iloc[i].name
            randomSelectedDict[i] = comb
            brickTagsetDict = OrderedDict()
            brickTagsetDict['point'] = str_conv(trueDF.loc[srcid]['BRICK_POINT']).split(',')
            brickTagsetDict['equip'] = str_conv(trueDF.loc[srcid]['BRICK_EQUIP_LIST']).split(',')
            brickTagsetDict['location'] = str_conv(trueDF.loc[srcid]['BRICK_LOCATION_LIST']).split(',')
            
            for tagsetType, brickTagsetList in brickTagsetDict.items():
                if brickTagsetList[0]=='':
                    unknownDict[tagsetType] += 1
                elif tagsetType in candSet.keys():
                    #if candSet[tagsetType] in brickTagsetList:
                    checkCorrectMapper = lambda aTagset:check_correct(candSet[tagsetType], aTagset)
                    if True in map(checkCorrectMapper, brickTagsetList):
                        correctCntDict[tagsetType] += 1
                    else:
                        incorrectCntDict[tagsetType] += 1
                    totalCntDict[tagsetType] += len(brickTagsetList)
                else:
                    totalCntDict[tagsetType] += len(brickTagsetList)
    recallDict = dict()
    precisionDict = dict()
    unknownRatioDict = dict()
    for tagsetType, correctCnt in correctCntDict.items():
        recallDict[tagsetType] = correctCnt / float(totalCntDict[tagsetType])
        precisionDict[tagsetType] = correctCnt / float(correctCnt+incorrectCntDict[tagsetType])
        unknownRatioDict[tagsetType] = unknownDict[tagsetType]/float(cnt)
    resultDict={'precision':precisionDict,\
               'recall':recallDict,
               'unknownRatioDict':unknownRatioDict,
                'sampleDict':randomSelectedDict
               }
    return resultDict

In [41]:
wordUpdateIterationNum = 0
combListDict = deepcopy(origCombListDict)
scoreDictDict = deepcopy(origScoreDictDict)
buildingScoreList = [calc_building_score(combListDict)]
resultHistory = list()
answerCnt = 0

In [ ]:
iterNum = 15

askFlag = False
initFlag = False
try:
    reduceCandNum = 0
    for iterCnt in range(0,iterNum):
        print wordUpdateIterationNum

        if initFlag:
            print "INITED"
            ######################### Init Cand 
            beginTime =  datetime.now()
            if mpFlag:
                pool = multiprocessing.Pool(workerNum)
                combListList = pool.map(init_comb, zip(sentenceDict.values(), repeat(subTagListDictList), repeat(scoreDictDict), repeat(0)))
                pool.terminate()
                pool.close()
                pool.join()
            else:
                combListList = map(init_comb, zip(sentenceDict.values(), repeat(subTagListDictList), repeat(scoreDictDict), repeat(0)))
                pass
            combListDict = dict((i,combList) for i, combList in zip(idxList,combListList))
            endTime = datetime.now()
            print "comb init time:", (endTime-beginTime).total_seconds()/60, 'minutes'


        #print iterCnt
        ########################## Update Weights
        beginTime =  datetime.now()
        makeDict = lambda x:dict([x])
        adder = lambda x,y:x+y
        dictgetter = lambda x:x.items()
        if mpFlag:
            pool = multiprocessing.Pool(workerNum)
            updatedDictList = pool.map(eval_weight, zip(map(makeDict,combListDict.items()), repeat(scoreDictDict)))
            pool.terminate()
            pool.close()
            pool.join()
            pass
        else:
            updatedDictList = map(eval_weight, zip(map(makeDict,combListDict.items()), repeat(scoreDictDict)))
        combListDict = dict(reduce(adder,map(dictgetter, updatedDictList)))

        endTime = datetime.now()
        print "weight computing time:", (endTime-beginTime).total_seconds()/60, '(min)'


        #################### Update Word Meaning
        print "init word update"

        baseBuildingScore = calc_building_score(combListDict)

        beginTime =  datetime.now()
        if mpFlag:
            pool = multiprocessing.Pool(workerNum)
            removeWordTagsetTupleList = pool.map(eval_word_meaning2, zip(repeat(combListDict), scoreDictDict.keys(), \
                                                                         repeat(scoreDictDict)))
            pool.terminate()
            pool.close()
            pool.join()
        else:
            removeWordTagsetTupleList = map(eval_word_meaning2, zip(repeat(combListDict), scoreDictDict.keys(), repeat(scoreDictDict)))
        removeWordTagsetDict = dict([el for el in removeWordTagsetTupleList if el])
        endTime = datetime.now()

        #update removing word meanings
        for word, tagset in removeWordTagsetDict.items():
            del scoreDictDict[word][tagset]

        combListDict = update_comb_cand_dict_score(combListDict, scoreDictDict)
        buildingScoreList.append(calc_building_score(combListDict))
        wordUpdateIterationNum += 1


        ################## Remove weak candidates
        combListDict = remove_weak_candidates(combListDict)
        #combListDict = remove_weak_candidates(combListDict)
        print "word meaning computing time:", (endTime-beginTime).total_seconds()/60, '(min)'
        print buildingScoreList
        print "================"

        ########## Calculate best candidates
        bestCandScoreDict = calc_best_cand_dict(combListDict)
        #scoreGetter = lambda x:x.get_score()
        #for i, combList in combListDict.items():
        #    scoreList = map(scoreGetter, combList)
        #    maxVal = max(scoreList)
        #    maxIdx = scoreList.index(maxVal)
        #    bestComb = combList[maxIdx]
        #    bestCandScoreDict[i] = (bestComb, bestComb.get_cand_set(), maxVal)

        resultHistory.append(eval_result(bestCandScoreDict, randIdxList, trueDF))


        if askFlag:
            askFlag, initFlag, answerNum = ask_experts(bestCandScoreDict, combListDict, scoreDictDict)
            answerCnt += answerNum

    print "Human Inputs: ", answerCnt
except:
    pool.terminate()
    pool.join()

1
weight computing time: 1.07244868333 (min)
init word update


In [45]:
result = resultHistory[-1]
sampleDict = result['sampleDict']
for i, comb in sampleDict.items():
    print i, comb.get_sentence(), comb.get_cand_set()
print result

163 ['nae', 'trunk', 'rhc', 'wc', 'adj', 'ap', 'rm', 'wc', 'adj', 'warm', 'cool', 'adjustment', 'temperature', 'sensor', 'meter'] {'location': u'room', 'point': u'warm_cool_adjust_sensor'}
10 ['nae', 'trunk', 'vav', 'aclg', 'stpt', 'ap', 'rm', 'aclg', 'stpt', 'actual', 'cooling', 'setpoint', 'temperature', 'sensor', 'meter'] {'location': u'room', 'point': u'cooling_temperature_setpoint'}
146 ['nae', 'trunk', 'vav', 'sup', 'flow', 'ap', 'rm', 'sup', 'flow', 'supply', 'air', 'flow', 'flow', 'sensor', 'meter'] {'location': u'room', 'point': u'supply_air_flow_sensor'}
47 ['nae', 'trunk', 'vav', 'zn', 'ap', 'rm', 'zn', 'zone', 'temperature', 'temperature', 'sensor', 'meter'] {'equip': u'vav', 'point': u'zone_temperature_sensor'}
82 ['nae', 'trunk', 'vav', 'zn', 'ap', 'rm', 'zn', 'zone', 'temperature', 'temperature', 'sensor', 'meter'] {'equip': u'vav', 'point': u'zone_temperature_sensor'}
59 ['nae', 'trunk', 'vav', 'ahtg', 'stpt', 'ap', 'rm', 'ahtg', 'stpt', 'actual', 'heating', 'setpoint',

In [130]:
for i in sampleDict.keys():
    srcid = sensorDF.index[i]
    brickPoint = (trueDF.loc[srcid]['BRICK_POINT']).split(',')
    tagset = resultHistory[-1]['sampleDict'][i].get_cand_set()['point']
    print brickPoint
    print tagset
    print tagset in brickPoint

['warm_cool_adjust_sensor']
warm_cool_adjust_sensor
True
['effective_cooling_temperature_setpoint']
cooling_temperature_setpoint
False
['supply_air_flow_sensor']
supply_air_flow_sensor
True
['zone_temperature_sensor']
zone_temperature_sensor
True
['zone_temperature_sensor']
zone_temperature_sensor
True
['effective_heating_temperature_setpoint']
room_temperature_setpoint
False


In [107]:
filename = 'result/'+buildingName+'_comb_cand.csv'
with open(filename, 'wb') as fp:
    writer = csv.writer(fp)
    writer.writerow(['sentence', 'true point type', 'true equip type', 'point correct?', 'candidate combi', 'score', 'candidate combi', 'score'])
    addFunc = lambda x,y:x+y

    candScoreGetter = lambda x:(x.get_cand_set(), x.get_score())
    
    for i, combList in combListDict.items():
        #candTupleList = reduce(addFunc,map(itemgetter(1), tagsetTypeBasedList),[])
        #candTupleList = sorted(candTupleList, key=itemgetter(1), reverse=True)
        candTupleList = map(candScoreGetter, combList)
        candTupleList = sorted(candTupleList, key=itemgetter(1), reverse=True)
        comb = combList[0]
        if not buildingName in notUcsdBuildings:
            srcid = sensorDF.index[i]
            trueRow = trueDF.loc[srcid]
            pointCorrect = check_correct(bestCandScoreDict[i][1]['point'], str_conv(trueRow['Schema Label']))
            writeList = [comb.get_sentence(), trueRow['Schema Label'], trueRow['Equipment Type'], 1 if pointCorrect else 0]
        else:
            label = df.iloc[i]['Schema Label']
            writeList = [comb.get_sentence(), label]
        for candTuple in candTupleList:
            writeList.append(candTuple[0])
            writeList.append(candTuple[1])
        writer.writerow(writeList)
f = gdrive.CreateFile()
f.SetContentFile(filename)
f.Upload()
f = None

In [108]:
filename = 'result/'+buildingName+'_random_sample_result.csv'
with open(filename, 'wb') as fp:
    writer = csv.writer(fp)
    writer.writerow(['srcid', 'sentence', 'BRICK_POINT', 'estimated_point', 'point correct?', \
                     'BRICK_EQUIP', 'estimated_equip', 'equip correct?', \
                     'BRICK_LOCATION', 'estimated_loc', 'loc correct?', 'cand_set'])
    for i, combList in combListDict.items():
        rowStr = list()
        if i in randIdxList:
            srcid = sensorDF.iloc[i].name
            rowStr.append(srcid)
            (bestComb, bestCandSet, bestScore) = bestCandScoreDict[i]
            rowStr.append(bestComb.get_sentence())
            brickTagsetDict = OrderedDict()
            brickTagsetDict['point'] = str_conv(trueDF.loc[srcid]['BRICK_POINT']).split(',')
            brickTagsetDict['equip'] = str_conv(trueDF.loc[srcid]['BRICK_EQUIP_LIST']).split(',')
            brickTagsetDict['location'] = str_conv(trueDF.loc[srcid]['BRICK_LOCATION_LIST']).split(',')
            for tagsetType, correctTagsetList in brickTagsetDict.items():
                rowStr.append(correctTagsetList)
                if tagsetType in bestCandSet.keys():
                    estimatedTagset = bestCandSet[tagsetType]
                    correct = 1 if estimatedTagset in correctTagsetList else 0
                    rowStr.append(estimatedTagset)
                else:
                    correct = 0
                    rowStr.append('')
                rowStr.append(correct)
            rowStr.append(bestCandSet)
            writer.writerow(rowStr)
            
                
f = gdrive.CreateFile()
f.SetContentFile(filename)
f.Upload()
f = None

In [109]:
### Transfer knowledge

foundWordDict = OrderedDict()
cnt = 0
for (comb, candSet, candSetScore) in sorted(bestCandScoreDict.values(), key=itemgetter(2), reverse=True):
    if cnt>coverageNum*0.2:
        break
    cnt +=1
    sentence = comb.get_sentence()
    for word in sentence:
        meaningTagsetList = scoreDictDict[word].keys()
        if len(meaningTagsetList)>1:
            continue
        meaningTagset = meaningTagsetList[0]
        correctFlag = True
        for meaningTag in meaningTagset.split():
            pointTagset = candSet['point'].split('_')
            if 'equip' in candSet.keys():
                equipTagset = candSet['equip'].split('_')
            else:
                equipTagset = []
            if meaningTag not in pointTagset+equipTagset:
                correctFlag = False
        if correctFlag:
            foundWordDict[word] = meaningTagset

In [43]:
print "Finished!!!"
from IPython.display import Audio
sound_file = 'etc/fins_success.wav'
Audio(url=sound_file, autoplay=True)

Finished!!!


In [111]:
combList = origCombListDict[143]
for comb in combList:
    print comb.get_cand_set()

{'point': u'zone_temperature_sensor'}
{'point': u'warmest_zone_temperature_sensor'}
{'point': u'highest_zone_temperature_sensor'}
{'point': u'coldest_zone_temperature_sensor'}
{'point': u'zone_air_temperature_sensor'}
{'equip': u'vav', 'point': u'zone_temperature_sensor'}
{'equip': u'vav', 'point': u'zone_air_temperature_sensor'}
{'equip': u'hvac', 'point': u'highest_zone_temperature_sensor'}
{'equip': u'vav', 'location': u'zone', 'point': u'zone_temperature_sensor'}
{'equip': u'vav', 'location': u'zone', 'point': u'zone_air_temperature_sensor'}
{'equip': u'hvac', 'location': u'zone', 'point': u'highest_zone_temperature_sensor'}
{'equip': u'vav', 'location': u'hvac_zone', 'point': u'zone_temperature_sensor'}
{'equip': u'vav', 'location': u'hvac_zone', 'point': u'zone_air_temperature_sensor'}
{'equip': u'hvac', 'location': u'hvac_zone', 'point': u'highest_zone_temperature_sensor'}
{'equip': u'vav', 'location': u'lighting_zone', 'point': u'zone_temperature_sensor'}
{'equip': u'vav', 'loc

In [112]:
print '['
for i in range(0,10):
    print int(random.random() * 2334), ','
    
print ']'

[
1514 ,
1582 ,
1164 ,
2131 ,
1836 ,
1401 ,
832 ,
779 ,
1561 ,
43 ,
]


In [113]:
# Evaluation!!!!!!!!!!!

# Evaluate scoring
sentenceIdxList = [
1609 ,
1317 ,
418 ,
321 ,
305 ,
283 ,
2274 ,
558 ,
813 ,
1289 ,
]
wordList = list()
for idx in sentenceIdxList:
    #print idx
    sentence = origSentenceDict[idx]
    wordList += sentence
wordList = list(set(wordList))
print wordList
if buildingName == 'ebu3b':
    evalWordMeaningDict = {'penthouse':'',
                           'code':'',
                           'feedback':'',
                           'ah':'ahu',
                           'setpoint': 'setpoint',
                           'supflow': 'supply flow',
                           'meter': 'meter',
                           'vma': '',
                           'actclg': 'effective cooling',
                           'cmd': 'command',
                           'bonner':'',
                           'n': '',
                           'occupied': 'occupied',
                           'sav':'vav',
                           'temperature':'temperature',
                           'nae':'',
                           'occ':'occupied',
                           'clg':'cooling',
                           'nd':'',
                           'efs':'exhaust fan',  ####TODO Check
                           'rd':'',
                           'th':'',
                           'rm':'room',
                           'bypdmp':'bypass damper',
                           'coomand':'command',
                           'common':'',
                           'ef':'exhaust fan',
                           'bypass':'bypass',
                           'fc':'',
                           'stpt':'setpoint',
                           'dx':'',
                           'demand':'',
                           'dmd':'',
                           'fec':'',
                           'a':'',
                           'c':'',
                           'actual':'',
                           'flr':'floor',
                           'unlock':'',
                           'roof':'',
                           't':'',
                           'st':'setpoint',
                           'sa':'supply air',
                           'com':'',
                           'vnd':'',
                           'stby':'standby',
                           'ebu':'',
                           'on':'',
                           'adj':'adjust',
                           'dmpr':'damper',
                           'pos':'position',
                           'acthtgsp':'effective heating temperature setpoint',
                           'actclgsp':'effective cooling temperature setpoint',
                           'sup':'supply',
                           'commonsp': 'setpoint',
                           'dpr':'damper',
                           
    }
elif buildingName=='bonner':
    evalWordMeaningDict = {'penthouse':'', 
                            'code':'', 
                            'feedback':'', 
                            'ah':'ahu', 
                            'supflow':'supply air flow',
                            'vma':'', 
                            'actclg':'effective cooling',
                            'cmd':'command', 
                            'bonner':'', 
                            'n':'', 
                            'sav':'vav', 
                            'nae':'',
                            'occ':'occupied', 
                            'clg':'cooling', 
                            'nd':'', 
                            'efs':'exhaust fan status', 
                            'rd':'', 
                            'th':'', 
                            'rm':'room', 
                            'bypdmp':'bypass damper', 
                            'coomand':'command',
                           'common':'',
                           'ef':'exhaust fan',
                           'bypass':'bypass',
                           'fc':'',
                           'stpt':'setpoint',
                           'dx':'',
                           'demand':'',
                           'dmd':'',
                           'fec':'',
                           'a':'',
                           'c':'',
                           'actual':'',
                           'flr':'floor',
                            'percent':'percent', 
                            'actual':'', 
                            'flr':'floor', 
                           'unlock':'',
                           'roof':'',
                           't':'', 
                           'st':'setpoint', 
                           'sa':'supply air', 
                           'com':'command'}
elif 'ap_m':
    evalWordMeaningDict = {'htg':'hetaing',
                           'int':'integral', 
                           'it':'integral', 
                           'vp':'', 
                           'ap':'',
                           'minimum':'min', 
                           'aclg':'effective cooling',
                           'ahtg':'effective heating', 
                           'sathtg': 'supply air heating', 
                           'parameters':'', 
                           'nae':'', 
                           'occ':'occupied', 
                           'rm':'room', 
                           'wc':'warm cool', 
                           'tuning':'', 'stpt':'setpoint', 'trunk':'', 'vlv':'valve', 'a':'', 'actual':'effective', 
                           'ohmin':'occupied heating min', 
                           'adj':'adjust', 'cmd':'command', 'm':'', 'n':'', 'omin':'occupied min', 
                           's':'', 'time':'', 'sa':'supply air', 'adjustment':'adjust'
                          }
    
for word in wordList:
    if word in tagList:
        evalWordMeaningDict[word] = word
    
evalScoreDictDict = dict()
for word, meaning in evalWordMeaningDict.items():
    evalScoreDictDict[word] = {meaning:1}
evalMaxScoreDict = calc_max_score_dict(evalScoreDictDict)

evalCombListDict = dict()
for idx in sentenceIdxList:
    sentence = origSentenceDict[idx]
    combList = init_comb((sentence, subTagListDictList, \
                          evalScoreDictDict, 0))
    evalCombListDict[idx] = combList
    

for i in range(0,10):
    evalCombListDict = eval_weight((evalCombListDict, evalScoreDictDict))

['htg', 'vav', 'int', 'flow', 'setpoint', 'it', 'meter', 'vp', 'ap', 'minimum', 'aclg', 'ahu', 'adjustment', 'ahtg', 'sathtg', 'temperature', 'parameters', 'nae', 'occupied', 'occ', 'valve', 'percent', 'cooling', 'air', 'supply', 'rm', 'cool', 'sensor', 'wc', 'tuning', 'pressure', 'warm', 'stpt', 'trunk', 'vlv', 'a', 'actual', 'ohmin', 'adj', 'cmd', 'm', 'heating', 'n', 'omin', 's', 'command', 'time', 'sa']


In [114]:
scoreGetter = lambda x:x.get_score()
for i, combList in evalCombListDict.items():
    bestComb = sorted(combList, key=scoreGetter, reverse=True)[0]
    print i, bestComb.get_sentence(), bestComb.get_cand_set()

321 ['nae', 'trunk', 'ahu', 'tuning', 'parameters', 'sathtg', 'it', 'ap', 'sathtg', 'it', 'supply', 'air', 'heating', 'int', 'time', 'command', 'setpoint'] {'point': u'heating_supply_air_temperature_integral_time_setpoint'}
2274 ['nae', 'trunk', 'vav', 'ohmin', 'stpt', 'ap', 'rm', 'ohmin', 'stpt', 'occupied', 'heating', 'minimum', 'flow', 'flow', 'command', 'setpoint'] {'equip': u'vav', 'location': u'room', 'point': u'occupied_heating_min_discharge_air_flow_setpoint'}
1317 ['nae', 'trunk', 'vav', 'wc', 'adj', 'ap', 'rm', 'wc', 'adj', 'warm', 'cool', 'adjustment', 'temperature', 'sensor', 'meter'] {'equip': u'vav', 'location': u'room', 'point': u'warm_cool_adjust_sensor'}
1609 ['nae', 'trunk', 'vav', 'wc', 'adj', 'ap', 'rm', 'wc', 'adj', 'warm', 'cool', 'adjustment', 'temperature', 'sensor', 'meter'] {'equip': u'vav', 'location': u'room', 'point': u'warm_cool_adjust_sensor'}
418 ['nae', 'trunk', 'vav', 'htg', 'vlv', 'ap', 'rm', 'htg', 'vlv', 'heating', 'valve', 'command', 'percent', 'co

In [115]:
combList = evalCombListDict[2274]
for comb in sorted(combList, key=scoreGetter,reverse=True):
    print comb.get_cand_set(), comb.get_score()

{'equip': u'vav', 'location': u'room', 'point': u'occupied_heating_min_discharge_air_flow_setpoint'} 0.445445718369
{'equip': u'vav', 'location': u'room', 'point': u'occupied_heating_min_supply_air_flow_setpoint'} 0.445445718369
{'equip': u'vav', 'location': u'server_room', 'point': u'occupied_heating_min_discharge_air_flow_setpoint'} 0.438681027517
{'equip': u'vav', 'location': u'server_room', 'point': u'occupied_heating_min_supply_air_flow_setpoint'} 0.438681027517
{'location': u'room', 'point': u'occupied_heating_min_discharge_air_flow_setpoint'} 0.43266396481
{'location': u'room', 'point': u'occupied_heating_min_supply_air_flow_setpoint'} 0.43266396481
{'equip': u'vav', 'point': u'occupied_heating_min_discharge_air_flow_setpoint'} 0.427109525649
{'equip': u'vav', 'point': u'occupied_heating_min_supply_air_flow_setpoint'} 0.427109525649
{'location': u'server_room', 'point': u'occupied_heating_min_discharge_air_flow_setpoint'} 0.423945634886
{'location': u'server_room', 'point': u'oc

In [116]:
assert(False)
import google_drive
reload(google_drive)
from google_drive import gdrive

Visit here: https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.install&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=842684053512-91e0897dkl1iuiuunb0mkfkdqddi41fl.apps.googleusercontent.com&access_type=offline


KeyboardInterrupt: 